In [1]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

In [2]:
# Configuration
class Config:
    PDF_DIRECTORY = "papers/"
    VECTOR_DB_PATH = "./vector_db"
    MODEL_NAME = "llama3.2"
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    SEARCH_K = 4

In [3]:
class ResearchAssistant:
    def __init__(self, config):
        self.config = config
        self.vectorstore = None
        self.qa_chain = None
        self.setup_embeddings()
    
    def setup_embeddings(self):
        """Configure the model embeddings"""
        self.embeddings = HuggingFaceEmbeddings(
            model_name=self.config.EMBEDDING_MODEL,
            model_kwargs={'device': 'cuda'},  # Use 'cuda' if you have a GPU, otherwise use 'cpu'
            encode_kwargs={'normalize_embeddings': True}
        )
        print(f"Embeddings configured: {self.config.EMBEDDING_MODEL}")
    
    def load_vectorstore(self, use_chroma=True):
        """Load an existent vectorstore"""
        if use_chroma:
            self.vectorstore = Chroma(
                persist_directory=self.config.VECTOR_DB_PATH,
                embedding_function=self.embeddings
            )
        else:
            self.vectorstore = FAISS.load_local(
                self.config.VECTOR_DB_PATH, 
                self.embeddings, 
                allow_dangerous_deserialization=True
            )
        print("Vector store loaded")
        return self.vectorstore
    
    def setup_qa_chain(self):
        """Configure the string question-answer"""
        if not self.vectorstore:
            raise ValueError("Create or load the vectorstore first")
        
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": self.config.SEARCH_K}
        )
        
        llm = Ollama(
            model=self.config.MODEL_NAME,
            temperature=0.1,
            num_ctx=4096
        )
        
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            return_source_documents=True
        )
        print("QA Chain configured correctly")
        return self.qa_chain
    
    def ask_question(self, question, verbose=False):
        """Make a question to the assistant"""
        if not self.qa_chain:
            raise ValueError("Configure QA chain first")
        
        result = self.qa_chain.invoke({"query": question})
        
        if verbose:
            print(f"Question: {question}")
            print(f"Answer: {result['result']}")
            print("\nConsulted Sources:")
            for i, doc in enumerate(result['source_documents']):
                print(f"{i+1}. {doc.metadata['source_file']} - Page {doc.metadata.get('page', 'N/A')}")
            print("=" * 60)
        
        return result

def interactive_chat(assistant):
    print("Interactive mode - Research assistant")
    print("Type 'quit' to exit or 'reset' to start a new conversation")
    print("=" * 60)
    
    while True:
        question = input("\nQuestion: ").strip()
        
        if question.lower() == 'quit':
            break
        if question.lower() == 'reset':
            print("Conversation reset")
            continue
        if not question:
            continue
        
        try:
            response = assistant.ask_question(question, verbose=True)
        except Exception as e:
            print(f"Error: {str(e)}")

In [4]:
config = Config()

# Initialize and Load
assistant = ResearchAssistant(config)
assistant.load_vectorstore(use_chroma=True)
assistant.setup_qa_chain()



C:\Users\Gerardo\AppData\Local\Temp\ipykernel_8696\3454659852.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Embeddings configured: sentence-transformers/all-MiniLM-L6-v2


C:\Users\Gerardo\AppData\Local\Temp\ipykernel_8696\3454659852.py:20: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectorstore = Chroma(


Vector store loaded
QA Chain configured correctly


C:\Users\Gerardo\AppData\Local\Temp\ipykernel_8696\3454659852.py:43: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=Ollama(model='llama3.2', num_ctx=4096, temperature=0.1), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000023B0D09EC80>, search_kwargs={'k': 4}))

In [5]:
interactive_chat(assistant)

Interactive mode - Research assistant
Type 'quit' to exit or 'reset' to start a new conversation



Question:  Does number of emitted photons depend on pump power?


Question: Does number of emitted photons depend on pump power?
Answer: Yes, the number of emitted photons does depend on pump power. According to the text, the emission rate at the highest pump power considered is 2.3 × 10^5 photon pairs per second for the pulsed-pump case and 5.7 × 10^8 photon pairs per second for the nondegenerate pulsed-pump regime, assuming the same average pump power and a repetition rate of 80 MHz. This suggests that an increase in pump power leads to an increase in the number of emitted photons.

Consulted Sources:
1. Conversion efficiency in the process of copolarized spontaneous four-wave mixing.pdf - Page 7
2. Conversion efficiency in the process of copolarized spontaneous four-wave mixing.pdf - Page 7
3. Energy correlations of photon pairs generated by a silicon microring resonator probed by Stimulated Four Wave Mixing.pdf - Page 3
4. Counter-propagating spontaneous four wave mixing photon-pair factorability and ultra-narrowband single photons.pdf - Page 5



Question:  Which materials are being used for research in photonic chips?


Question: Which materials are being used for research in photonic chips?
Answer: The text mentions the following materials being used for research in photonic chips:

1. Silicon
2. III–V semiconductors
3. Lithium niobate (LN)
4. Silicon carbide (SiC)
5. Nitrides
6. Diamond
7. Tantalum pentoxide (Ta2O5)

Additionally, the text also mentions "vð2Þ materials" which are likely referring to second-order nonlinear materials, but it does not specify what these materials are.

Consulted Sources:
1. 2022 Roadmap on integrated quantum photonics.pdf - Page 9
2. 2022 Roadmap on integrated quantum photonics.pdf - Page 7
3. On-chip heralded single photon sources.pdf - Page 24
4. 2022 Roadmap on integrated quantum photonics.pdf - Page 4



Question:  quit
